In [1]:
import sys
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD, RMSprop
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

Using TensorFlow backend.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# load train and test dataset
def load_dataset():
    with tf.device('/gpu:0'):
    	# load dataset
    	(trainX, trainY), (testX, testY) = cifar10.load_data()
    	# one hot encode target values
    	trainY = to_categorical(trainY)
    	testY = to_categorical(testY)
    	return trainX, trainY, testX, testY

In [3]:
# scale pixels
def prep_pixels(train, test):
    with tf.device('/gpu:0'):
    	# convert from integers to floats
    	train_norm = train.astype('float32')
    	test_norm = test.astype('float32')
    	# normalize to range 0-1
    	train_norm = train_norm / 255.0
    	test_norm = test_norm / 255.0
    	# return normalized images
    	return train_norm, test_norm

In [4]:
# define cnn model
def define_model():
    with tf.device('/gpu:0'):
      model = Sequential()
      model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
      model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
      model.add(MaxPooling2D((2, 2)))
      model.add(Flatten())
      model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
      model.add(Dense(10, activation='softmax'))
    	# compile model
      opt = RMSprop(lr=0.0001, decay=1e-6)
      model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
      return model

In [5]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
   with tf.device('/gpu:0'):
    	# plot loss
    	plt.subplot(211)
    	plt.title('Cross Entropy Loss')
    	plt.plot(history.history['loss'], color='blue', label='train')
    	plt.plot(history.history['val_loss'], color='orange', label='test')
    	# plot accuracy
    	plt.subplot(212)
    	plt.title('Classification Accuracy')
    	plt.plot(history.history['accuracy'], color='blue', label='train')
    	plt.plot(history.history['val_accuracy'], color='orange', label='test')
    	# save plot to file
    	filename = sys.argv[0].split('/')[-1]
    	plt.savefig(filename + '_plot.png')
    	plt.close()

In [7]:
# run the test harness for evaluating a given model.
# It is better to use a separate validation dataset, e.g. by splitting the train dataset 
# into train and validation sets. We will not split the data in this case, and instead use 
# the test dataset as a validation dataset to keep the example simple.
# This function involves all previously defined functions.
def run_test_harness():
    with tf.device('/gpu:0'):
       # load dataset
       print("Loading dataset...")
       trainX, trainY, testX, testY = load_dataset()
       # prepare pixel data
       print("Preparing pixel data...")
       trainX, testX = prep_pixels(trainX, testX)
       # define model
       print("Defining model...")
       model = define_model()
       # fit model
       print("Fitting model...")
       history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0)   	
       # evaluate model
       print("Evaluating model...")
       acc = model.evaluate(testX, testY, verbose=1)
       print('accuracy > %.3f' % (acc * 100.0))
       # learning curves
       summarize_diagnostics(history)

In [8]:
# entry point, run the test harness.
# This test harness can evaluate any CNN models we may wish to evaluate on the 
# CIFAR-10 dataset and can run on the CPU or GPU.
run_test_harness()

Loading dataset...
Preparing pixel data...
Defining model...
Fitting model...


KeyboardInterrupt: 